# LangChain Expression Language (LCEL) 🔗 

## Introduction
This notebook demonstrates the `use of the LangChain Expression Language (LCEL) to create and run various chains using LangChain's capabilities`. We will cover the setup, simple chains, more complex chains, and using OpenAI functions. Additionally, we will discuss handling fallbacks and improving output readability.

## Setup
First, we need to import the necessary libraries and set up the environment.

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

**Note** Ensure you have the required packages installed:

In [ ]:
#!pip install pydantic==1.10.8

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

We will start with a simple chain that generates a joke based on a given topic.

In [4]:
# Define the prompt template
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

# Initialize the model and output parser
model = ChatOpenAI()
output_parser = StrOutputParser()

C:\Users\Michela\AppData\Local\Temp\ipykernel_4636\2159996570.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [5]:
# Create the chain
chain = prompt | model | output_parser

In [6]:
# Invoke the chain with a topic
chain.invoke({"topic": "bears"})

"Why don't bears like fast food? Because they can't catch it!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.We will now create a more complex chain using a retriever to supply user-provided inputs to the prompt.

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [8]:
# Create a vector store from texts
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

C:\Users\Michela\AppData\Local\Temp\ipykernel_4636\1824015626.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings()
c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
# Retrieve relevant documents based on a query1
retriever.get_relevant_documents("where did harrison work?")

C:\Users\Michela\AppData\Local\Temp\ipykernel_4636\3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [10]:
# Retrieve relevant documents based on a query2
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

- Creating the prompt template for a more complex chain:

In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
# Define the runnable map and chain
from langchain.schema.runnable import RunnableMap

In [13]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [14]:
# Invoke the complex chain
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [15]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [16]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

### Bind and Using OpenAI Functions

We will demonstrate `how to bind and use OpenAI functions within LangChain`.

- Define the OpenAI Functions

In [17]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [18]:
# Define the prompt and runnable
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [19]:
runnable = prompt | model

In [42]:
# Invoke the model with a query
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 97, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7f029bc0-1ce3-4cce-9146-0d5a6d1391e2-0')

In [43]:
import json

response = runnable.invoke({"input": "what is the weather in sf"})

# Check if response is a dictionary (JSON-like)
if isinstance(response, dict):
    print(json.dumps(response, indent=4))  # Pretty-print JSON
else:
    print(response)  # Print directly if it's a string or another type


content='' additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 97, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-bf6bce0d-1c79-42aa-8b1b-01b1e722ebe6-0'


**Explanation output** It looks like the output from runnable.invoke() is a structured response, but it's printed in a raw, unformatted way. You can extract and print it nicely in a structured format.

In [44]:
# Pretty-Print the Response
import json

response = {
    "content": "",
    "additional_kwargs": {
        "function_call": {
            "arguments": '{"airport_code":"SFO"}',
            "name": "weather_search"
        }
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 17,
            "prompt_tokens": 97,
            "total_tokens": 114,
            "completion_tokens_details": {
                "accepted_prediction_tokens": 0,
                "audio_tokens": 0,
                "reasoning_tokens": 0,
                "rejected_prediction_tokens": 0
            },
            "prompt_tokens_details": {
                "audio_tokens": 0,
                "cached_tokens": 0
            }
        },
        "model_name": "gpt-3.5-turbo",
        "system_fingerprint": None,
        "finish_reason": "function_call",
        "logprobs": None
    },
    "id": "run-567ec960-134a-498a-b1de-9eb77e133021-0"
}

# Pretty-printing with indentation
print(json.dumps(response, indent=4))


{
    "content": "",
    "additional_kwargs": {
        "function_call": {
            "arguments": "{\"airport_code\":\"SFO\"}",
            "name": "weather_search"
        }
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 17,
            "prompt_tokens": 97,
            "total_tokens": 114,
            "completion_tokens_details": {
                "accepted_prediction_tokens": 0,
                "audio_tokens": 0,
                "reasoning_tokens": 0,
                "rejected_prediction_tokens": 0
            },
            "prompt_tokens_details": {
                "audio_tokens": 0,
                "cached_tokens": 0
            }
        },
        "model_name": "gpt-3.5-turbo",
        "system_fingerprint": null,
        "finish_reason": "function_call",
        "logprobs": null
    },
    "id": "run-567ec960-134a-498a-b1de-9eb77e133021-0"
}


- Improving Readability with Rich

Using the rich library to improve the readability of nested dictionary outputs.

In [ ]:
%pip install rich

In [50]:
from rich import print
from rich.pretty import Pretty

response = runnable.invoke({"input": "what is the weather in sf"})
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 97,
            'total_tokens': 114,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-33ce8a2a-c11a-4c93-a594-6097e1c0ca86-0'
)

**Explanation output** As it can been seen from the above output, this makes it easier to read complex nested dictionaries! 

In [21]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

- Bind the function to the model

In [22]:
# Bind the functions to the model
model = model.bind(functions=functions)

In [23]:
runnable = prompt | model

In [45]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7372d73d-a15f-4706-a59d-343c7b375d34-0')

In [46]:
import json

response = runnable.invoke({"input": "how did the patriots do yesterday?"})

# Check if response is a dictionary (JSON-like)
if isinstance(response, dict):
    print(json.dumps(response, indent=4))  # Pretty-print JSON
else:
    print(response)  # Print directly if it's a string or another type

content='' additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-ff8f97c5-50ab-4964-b0df-95022d7bbc4f-0'


**Explanation output** It looks like the output from runnable.invoke() is a structured response, but it's printed in a raw, unformatted way. You can extract and print it nicely in a structured format.

In [47]:
# Code to Pretty-Print the Response
import json

# Invoke the model
response = runnable.invoke({"input": "what is the latest update on the New England Patriots?"})

# Check if response is a dictionary before formatting
if isinstance(response, dict):
    print(json.dumps(response, indent=4, ensure_ascii=False))
else:
    print(response)  # Print as-is if not a dictionary


content='' additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 102, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-9167003c-7c53-4dea-a440-df341749a9b3-0'


In [49]:
from rich import print
from rich.pretty import Pretty

response = runnable.invoke({"input": "what is the latest update on the New England Patriots?"})
print(Pretty(response))


AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 19,
            'prompt_tokens': 102,
            'total_tokens': 121,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-b999337d-ae5e-4821-8304-b6ee9ba2cd61-0'
)

### Handling Fallbacks

In [25]:
from langchain.llms import OpenAI
import json

In [26]:
# Initialize the fallback model
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

C:\Users\Michela\AppData\Local\Temp\ipykernel_4636\2498289737.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(


In [27]:
# Define a challenge
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [28]:
# Invoke the fallback chain
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [29]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [51]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [52]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose is a rose is a rose'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

In [53]:
from rich import print
from rich.pretty import Pretty

response = chain.invoke(challenge)
print(Pretty(response))

{
    'poem1': {
        'title': 'The Rose',
        'author': 'Emily Dickinson',
        'firstLine': 'A rose by any other name would smell as sweet'
    },
    'poem2': {
        'title': 'The Road Not Taken',
        'author': 'Robert Frost',
        'firstLine': 'Two roads diverged in a yellow wood'
    },
    'poem3': {
        'title': 'Hope is the Thing with Feathers',
        'author': 'Emily Dickinson',
        'firstLine': 'Hope is the thing with feathers that perches in the soul'
    }
}

In [54]:
final_chain = simple_chain.with_fallbacks([chain])

In [55]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [56]:
from rich import print
from rich.pretty import Pretty

response = final_chain.invoke(challenge)
print(Pretty(response))

{
    'poem1': {'title': 'The Rose', 'author': 'Emily Dickinson', 'firstLine': 'A rose is a rose is a rose'},
    'poem2': {
        'title': 'The Road Not Taken',
        'author': 'Robert Frost',
        'firstLine': 'Two roads diverged in a yellow wood'
    },
    'poem3': {
        'title': 'Hope is the Thing with Feathers',
        'author': 'Emily Dickinson',
        'firstLine': 'Hope is the thing with feathers'
    }
}

## Interface

In this context, the term "interface" ilikely refers to the way users interact with the LangChain library to build and run various chains. This involves defining prompts, models, and chains, and then invoking these chains with specific inputs to get desired outputs.  
Here the interface is the `set of functions and classes provided by LangChain`, such as `ChatPromptTemplate, ChatOpenAI, StrOutputParser, and RunnableMap`. These components allow users to define and execute chains in a structured and modular way.These components work together to provide a flexible and powerful interface for building and executing chains, enabling users to define complex workflows in a modular and readable manner.

In [34]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [35]:
chain.invoke({"topic": "bears"})

"Why did the bear break up with his girlfriend?\nBecause she couldn't bear his clingy behavior!"

In [36]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!",
 'Why was the frog always so happy? Because he eats whatever bugs him!']

In [37]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 bring
 a
 ladder
 to
 the
 bar
?
 


Because
 he
 heard
 the
 drinks
 were
 on
 the
 house
!



**Explanation output** a joke formatted in a humorous way with additional spaces between each word. The joke goes:  
Q: Why did the bear bring a ladder to the bar?  
A: Because he heard the drinks were on the house!  

The punchline "the drinks were on the house" is a play on words, meaning both that the drinks are free (a common idiom) and that the drinks are literally located on the roof, hence the need for a ladder.

`The format with extra spaces between each word` can have several advantages such as `emphasis and clarity`, ``visual impact`, `engagement` and `humor`.

In [38]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## Conclusion
This notebook demonstrated the use of LangChain Expression Language (LCEL) to create and run various chains. We covered setup, simple and complex chains, using OpenAI functions, improving readability with the rich library, and handling fallbacks. These examples illustrate the flexibility and power of LangChain for building and running chains of different complexities.

Here's a short description of the interface as used in the notebook:  

Interface in LangChain Notebook
The LangChain notebook demonstrates how to use the LangChain interface to create and execute chains of operations. This interface includes:  
- Prompt Templates: ChatPromptTemplate allows users to define prompts with placeholders for dynamic content.
- Models: ChatOpenAI represents the language model used to process prompts and generate responses.
- Output Parsers: StrOutputParser parses the model's output into a desired format.
= Runnable Maps: RunnableMap allows for the creation of more complex chains by mapping inputs to specific functions or operations.